## Logs
- Faithfulness NLI
    - Without CoT
    - With CoT ( WIN)  
    - WikiQA 
        - generated non factual answer for measuring faithfulness agreement.
        - Kendall Score = 0.7
    - HotPotQA
        - Accuracy = 0.75 
    - Possible Improvements 
        - improve statement generation

- Relevance scores
    - QGen method
        - models tried : t5-base / gptneo-125M
        - WikiQA
            - Kendall score = 0.65
            - observations : finetune model on prompt/answer pairs to improve performance.
    - Cross-encoder method
        - models tried : distilbert 
        - WikiQA
            - kendall score = 0.63
            

In [2]:
%load_ext autoreload

The autoreload module is not an IPython extension.


In [3]:
import json
from datasets import load_dataset
import re
import os
import openai
from tqdm import tqdm
import numpy as np
import random
from scipy.stats import kendalltau, spearmanr

In [4]:
os.chdir("/Users/shahules/belar/src/")

## OpenAI API

In [5]:
os.environ["OPENAI_API_KEY"] = json.load(open("/Users/shahules/openai-key.json"))[
    "ikka"
]

In [17]:
import os
from openai import OpenAI

client = OpenAI()


In [18]:
def llm(prompt, **kwargs):
    response = client.chat.completions.create(
        model=kwargs.get("model", "gpt-3.5-turbo"),
        messages=[{"role": "system", "content": prompt}],
        temperature=kwargs.get("temperature", 0),
        top_p=kwargs.get("top_p", 1),
        frequency_penalty=kwargs.get("frequency_penalty", 0.0),
        presence_penalty=kwargs.get("presence_penalty", 0.0),
        max_tokens=kwargs.get("max_tokens", 500),
        n=kwargs.get("n", 1),
    )
    return response

In [19]:
def json_logger(data, filename="nli_check"):
    output = json.load(open(filename + ".json"))
    output.append(data)
    with open(filename + ".json", "w") as file:
        json.dump(output, file, indent=4)

## Datasets

In [10]:
wikiqa_ragas = load_dataset("explodinggradients/ragas-wikiqa")

Extracting data files: 100%|█████| 1/1 [00:00<00:00, 304.69it/s]


Dataset parquet downloaded and prepared to /Users/shahules/.cache/huggingface/datasets/explodinggradients___parquet/explodinggradients--ragas-wikiqa-5b5116e5cb909aca/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


100%|████████████████████████████| 1/1 [00:00<00:00, 178.25it/s]


## Correlation

In [124]:
def get_corr(targets, predictions):
    scores = [kendalltau(x, y).correlation for x, y in zip(targets, predictions)]
    return [score if not np.isnan(score) else 0 for score in scores]

## QA-QG paradigm
- Generate question and answer pair from `generated answer`.
- Given `context`, ask these questions
- Verify answer correctness

In [11]:
Question_generation = """Given a text, extract {} noun phrases and create questions for each based on given text.
text: Albert Einstein was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. Best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics.
A: Germany
Q: Where was Albert Einstein born?
A: theory of relativity
Q: What is Albert Einstein best known for?
text: {}
"""

Question_answering = """Given a text and set of questions, answer the questions
text: Albert Einstein was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. Best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics.
questions: Where was Albert Einstein born?\n\nWhat is Albert Einstein best known for?
answers:Germany\n\ntheory of relativity
text: {}
questions:{}
answers:"""

Answer_verification = """Given a set of questions, correct answer and student's answer return the number of questions incorrectly answered by student.
Where was Albert Einstein born?\nCorrect answer: Germany\nStudent answer:India\n\n
What is Albert Einstein best known for?\nCorrect answer:  theory of relativity\nStudent answer: theory of relativity\n\n
Number of incorrect answers:1
{}
Number of incorrect answers:"""

In [12]:
def QAQG_fun(question, context, answer):
    """
    returns number of factual inconsistencies.
    """

    def answer_ver(qstn, answer, cand):
        return f"{qstn}\nCorrect answer: {answer}\nStudent answer: {cand}"

    num = len(answer.split(".")) - 1
    prompt = Question_generation.format(num, answer)
    output = llm(prompt)
    qa_pairs = [
        re.sub(r"A:|Q:", "", x).strip()
        for item in output["choices"][0]["text"].strip().split("\n\n")
        for x in item.split("\n")
    ]
    qa_pairs = [tuple(qa_pairs[i : i + 2]) for i in range(0, len(qa_pairs), 2)]
    print(qa_pairs)
    questions = "\n\n".join([qstn for ans, qstn in qa_pairs])
    prompt = Question_answering.format(context, questions)
    answers = llm(prompt)["choices"][0]["text"].split("\n\n")

    prompt = "\n\n".join(
        [answer_ver(qstn, ans, cand) for (ans, qstn), cand in zip(qa_pairs, answers)]
    )
    output = llm(Answer_verification.format(prompt))["choices"][0]["text"].strip()
    return int(output)

In [20]:
answer = "The actress who played Lolita, Sue Lyon, was 14 at the time of filming."
question = "What was the age of Sue Lyon when she played Lolita?"
context = """
Lolita is a 1962 psychological comedy-drama film[5] directed by Stanley Kubrick and based on the 1955 novel of the same title by Vladimir Nabokov, who is also credited with writing the screenplay. The film follows Humbert Humbert, a middle-aged literature lecturer who becomes sexually infatuated with Dolores Haze (nicknamed "Lolita"), a young adolescent girl. It stars James Mason, Shelley Winters, Peter Sellers and, as the titular character, Sue Lyon.

Owing to restrictions imposed by the Motion Picture Production Code, the film toned down the most provocative aspects of the novel, sometimes leaving much to the audience's imagination. The actress who played Lolita, Sue Lyon, was 14 at the time of filming."""

In [17]:
QAQG_fun(question, context, answer)

[('Sue Lyon', 'Who played the role of Lolita in the movie?')]


0

In [22]:
llm2([Question_generation.format(2, answer)])

<OpenAIObject chat.completion id=chatcmpl-7aiRcMcfrtt4jp5AK9PaIBMqFdlIB at 0x7f9b1b66e630> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "A: Lolita\nQ: What character did Sue Lyon play in the movie?\nA: 14\nQ: How old was Sue Lyon when she filmed Lolita?",
        "role": "assistant"
      }
    }
  ],
  "created": 1688985008,
  "id": "chatcmpl-7aiRcMcfrtt4jp5AK9PaIBMqFdlIB",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 35,
    "prompt_tokens": 128,
    "total_tokens": 163
  }
}

## G-Eval
- Define criterions to evaluate model.
- Normalize `score = prob(s) * s`

In [138]:
relevence = """
Evaluation Criteria.\n
Relevance (1-5) - how relevant is the reply to the given question.
1. Read the reply and compare it to the question. Check if the given reply
actually answers the question, and if it presents them in a clear and logical order.
2. The reply should include only required information to answer the question.
3. Penalize replies that contain redundancies and excess information.
4. Assign a score for Relevance on a scale of 1 to 5, where 1 is the lowest and
5 is the highest based on the Evaluation Criteria.

question:{}
reply:{}
score:"""

faithfulness = """
Evaluation Criteria.\n
Faithfulness (1-5) - how factually consistant is the reply with the given context.
1. Read the reply and compare it to the question. Check if the given reply
actually answers the question correctly, and if the reply is factualy consistent with the context.
2. Assign a score for faithfulness on a scale of 1 to 5, where 1 is the lowest and
5 is the highest based on the Evaluation Criteria.

context: {}
question:{}
reply:{}
score:"""

In [156]:
def gpt_faithfulness(question: list, context: list, answer: list):
    prompt = [
        faithfulness.format(c, q, a) for c, q, a in zip(question, context, answer)
    ]
    output = [output for output in llm(prompt)["choices"]]
    scores = [(out["text"].strip()) for out in output]
    scores = [
        int(score) if score in ["1", "2", "3", "4", "5"] else 1 for score in scores
    ]
    return scores


def gpt_relevance(question: list, answer: list):
    prompt = [relevence.format(q, a) for q, a in zip(question, answer)]
    output = [output for output in llm(prompt)["choices"]]
    scores = [(out["text"].strip()) for out in output]
    scores = [
        int(score) if score in ["1", "2", "3", "4", "5"] else 1 for score in scores
    ]
    return scores

In [89]:
def g_eval(question, context, answer):
    prompt = relevence.format(question, answer)
    output = llm(prompt)["choices"][0]
    prob = np.exp(sum(output["logprobs"]["token_logprobs"]))
    score = int(output["text"].strip())
    print(score)
    return prob * score

In [90]:
question = "Which year did Lolita release?"
answer = "Lolita film released in 1947."

In [54]:
gpt_relevance(question, answer)

5

In [151]:
q, a, c = (
    wikiqa_ragas["train"][0]["question"],
    wikiqa_ragas["train"][0]["generated_without_rag"],
    wikiqa_ragas["train"][0]["context"],
)

In [152]:
gpt_faithfulness([q], [c], [a])

[4]

In [91]:
gpt_relevance([q], [a])

[4]

## Relevancy Score 
- Scores `answers` according to `prompt`


### QGen scoring method

In [35]:
from ragas.metrics.answer_relevance import QGen

In [36]:
t5_qgen = QGen("t5-base", "cpu")

In [146]:
def predict_(examples):
    scores = {}
    questions = examples["question"]
    context = examples["context"]
    for col in COLUMNS:
        passage = examples[col]
        inputs = list(zip(questions, passage))
        # scores[f"{col}_relevance"] = t5_qgen.predict(inputs, show_progress=False)
        scores[f"{col}_relevance"] = gpt_faithfulness(questions, context, passage)
    return scores

- We assume `generated_with_rag > correct_answer > incorrect_answer` for relevancy.

In [142]:
COLUMNS = ["generated_with_rag", "correct_answer", "incorrect_answer"]

In [139]:
output = wikiqa_ragas["train"].map(predict_relevance, batched=True, batch_size=10)

In [ ]:
predictions = [[item[f"{k}_relevance"] for k in COLUMNS] for item in output]
target = [[2, 1, 0] for i in range(len(output))]
np.mean(get_corr(target, predictions))

Relevance

- 0.6337284370533437 for wikiQA gpt 3.5

- 0.6831823238905629 For wikiwa t5

## Faithfulness

In [157]:
COLUMNS = ["generated_with_rag", "correct_answer", "generated_without_rag"]

In [158]:
output = wikiqa_ragas["train"].map(predict_relevance, batched=True, batch_size=10)

In [159]:
predictions = [[item[f"{k}_facuality"] for k in COLUMNS] for item in output]
target = [[2, 1, 0] for i in range(len(output))]
np.mean(get_corr(target, predictions))

0.48110338184466117

0.48110338184466117 for GPT3.5

### Cross encoder method

In [17]:
from ragas.metrics.context_relevance import context_relavancy

In [24]:
def predict_relevance(examples):
    scores = {}
    questions = examples["question"]
    for col in COLUMNS:
        passage = examples[col]
        inputs = list(zip(questions, passage))
        scores[f"{col}_relevance"] = cross_encoder.predict(inputs, show_progress=False)
    return scores

In [ ]:
output = (
    wikiqa_ragas["train"]
    .select(range(0, 10))
    .map(predict_relevance, batched=True, batch_size=4)
)

In [ ]:
predictions = [[item[f"{k}_relevance"] for k in COLUMNS] for item in output]
target = [[2, 1, 0] for i in range(len(output))]
get_tau(target, predictions)

## Faithfulness on HotpotQA


In [134]:
import experimental

In [135]:
from importlib import reload

reload(experimental)

<module 'experimental' (namespace)>

In [136]:
from experimental.nli import NLI

In [10]:
hotpot_qa = load_dataset(
    "hotpot_qa",
    "distractor",
    split="validation",
).select(range(0, 20))

Found cached dataset hotpot_qa (/Users/shahules/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


In [138]:
false_answer_prompt = """Given a question and correct answer, generate an incorrect answer
question: Were Scott Derrickson and Ed Wood of the same nationality?
correct answer: yes
answer: no
question: {}
correct answer: {}
answer:"""


def generate_false_answers(question, answer):
    answer = llm(false_answer_prompt.format(question, answer))["choices"][0][
        "text"
    ].strip()
    return {"false_answer": answer}

In [139]:
hotpot_qa = hotpot_qa.map(lambda x: generate_false_answers(x["question"], x["answer"]))

Loading cached processed dataset at /Users/shahules/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5/cache-593e03a966a13563.arrow


In [53]:
def get_context(item):
    titles, ids = item["supporting_facts"].values()
    title_ids = [item["context"]["title"].index(i) for i in titles]
    sentences = [
        item["context"]["sentences"][i][k]
        for i, k in zip(title_ids, item["supporting_facts"]["sent_id"])
    ]
    orig_context = " ".join(sentences)
    return {"answer_context": orig_context}

In [54]:
hotpot_qa = hotpot_qa.map(lambda x: get_context(x), batched=False)

In [142]:
def predict_faithfulness(examples, scoring_fun=NLI.score):
    scores = {}
    questions = examples["question"]
    contexts = examples["answer_context"]
    for col in COLUMNS:
        answers = examples[col]
        while True:
            try:
                scores[f"{col}_factual"] = scoring_fun(questions, contexts, answers)
            except Exception as e:
                print(e)
                continue
            break
    return scores

In [143]:
COLUMNS = ["answer", "false_answer"]
hotpot_qa = hotpot_qa.map(predict_faithfulness, batched=True, batch_size=8)

Loading cached processed dataset at /Users/shahules/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5/cache-d51f81546b2858f1.arrow


In [164]:
predictions = [[item[f"{k}_factual"] for k in COLUMNS] for item in hotpot_qa]
target = [[1, 0] for i in range(len(hotpot_qa))]
incorrect = [
    idx for idx, item in enumerate(predictions) if all(np.argsort(item) != [1.0, 0.0])
]
print("Accuracy", 1 - (len(incorrect) / len(target)))

Accuracy 0.75


## Context relevancy

In [11]:
def get_all_facts(item):
    all_facts = item["context"]["sentences"]
    all_facts = [sent for para in all_facts for sent in para]
    return {"full_context": "".join(all_facts)}


hotpot_qa = hotpot_qa.map(get_all_facts, batched=False)

In [73]:
question_ex1 = "Were Scott Derrickson and Ed Wood of the same nationality?"
context_ex1 = 'Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer He lives in Los Angeles, California He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange"Tyler Bates is an American musician, music producer, and composer for films, television, and video games. Adam Collis is an American filmmaker and actor.Conrad Brooks is an American actor.Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.'
answer_ex1 = "Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer. \nEdward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director."

In [16]:
Context_relevency = """
Task: Candidate sentence extraction.
Given the question and context, extract minimum number of sentences from context required to answer the question. If the context do not contain information required to answer the question return "No candidate sentences found".

question: Which equation is known as worlds most famous equation?
context:\nAlbert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist,[5] widely ranked among the greatest and most influential scientists of all time. Best known for developing the theory of relativity, he also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century.
His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world's most famous equation".
sentences:His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world's most famous equation".

question: Were Scott Derrickson and Ed Wood of the same nationality?
context :\nScott Derrickson (born July 16, 1966) is an American director, screenwriter and producer He lives in Los Angeles, California He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange"Tyler Bates is an American musician, music producer, and composer for films, television, and video games. Adam Collis is an American filmmaker and actor.Conrad Brooks is an American actor.Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.
Now given a question and context, extract the minimum number of sentences from the given context required to answer the question completely. 
sentences:Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer. Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.

question:{}
context:\n{}
sentences:"""

In [179]:
i = 15
q, c = hotpot_qa[i]["question"], hotpot_qa[i]["full_context"]

In [183]:
c = "A black hole is a region of spacetime where gravity is so strong that nothing, including light or other electromagnetic waves, has enough energy to escape it.[2] The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole.[3][4] The boundary of no escape is called the event horizon. Although it has a great effect on the fate and circumstances of an object crossing it, it has no locally detectable features according to general relativity.[5] In many ways, a black hole acts like an ideal black body, as it reflects no light"

In [349]:
q = "what is general relativity?"
n = 2

In [300]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia(
    language="en", extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("Black hole")


def get_page_section(page, section):
    all_text = ""
    p_wiki = wiki_wiki.page(page)
    sections = p_wiki.sections_by_title(section)
    for s in sections:
        all_text += s.full_text()
    return all_text

In [21]:
from typing import List
from itertools import combinations
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/stsb-TinyBERT-L-4")


def sent_tokenize(sent):
    return [s[:-1] if s.endswith(".") else s for s in sent.strip().split(". ")]


class SentenceAgreement:
    def __init__(self, scoring="bert_score"):
        self.scoring = scoring

    @staticmethod
    def bert_score(para1, para2):
        sentences1, sentences2 = sent_tokenize(para1), sent_tokenize(para2)
        scores = cross_encoder.predict(list(itertools.product(sentences1, sentences2)))
        scores = scores.reshape(len(sentences1), len(sentences2))
        return scores.max(axis=1).mean()

    @staticmethod
    def jaccard_score(para1, para2):
        sentences1, sentences2 = sent_tokenize(para1), sent_tokenize(para2)
        intersect = len(np.intersect1d(sentences1, sentences2))
        union = len(np.union1d(sentences1, sentences2))
        return intersect / union

    def evaluate(self, answers: List[List[str]]):
        """
        eval nC2 combinations
        """
        scores = []
        groups = combinations(answers, 2)
        for group in groups:
            if self.scoring == "jaccard":
                score = self.jaccard_score(*group)
            elif self.scoring == "bert_score":
                score = self.bert_score(*group)
            scores.append(score)
        return np.mean(scores)

In [22]:
class ContextRelevacy:
    def __init__(self, strictness=2, agreement_metric="bert_score"):
        self.strictness = strictness
        self.sent_agreement = SentenceAgreement(agreement_metric)

    def score(self, question, context):
        scores = []
        outputs = llm(Context_relevency.format(q, c), n=self.strictness, temperature=1)
        outputs = [
            outputs["choices"][i]["text"].strip() for i in range(self.strictness)
        ]
        context_sents = sent_tokenize(context)
        for output in outputs:
            indices = [
                context.find(sent)
                for sent in sent_tokenize(output)
                if context.find(sent) != -1
            ]
            scores.append(len(indices) / len(context_sents))

        if self.strictness > 1:
            agr_score = self.sent_agreement.evaluate(outputs)
        else:
            agr_score = 1
        return agr_score * np.mean(scores)

In [491]:
c = get_page_section("HIV/AIDS", "Prevention")
c = " ".join(c.split(" ")[:500])
q = "When was the first HIV case detected?"

In [501]:
output = llm(
    [
        Context_relevency.format(q, c),
        Context_relevency.format("How to prevent AIDS?", c),
    ],
    n=n,
    temperature=1,
)

In [1]:
from ragas.metrics import context_relevancy

/opt/anaconda3/envs/blade2blade/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

In [3]:
context_relevancy.init_model()

In [4]:
dataset = load_dataset("explodinggradients/fiqa", "ragas_eval")

Found cached dataset fiqa (/Users/shahules/.cache/huggingface/datasets/explodinggradients___fiqa/ragas_eval/1.0.0/3dc7b639f5b4b16509a3299a2ceb78bf5fe98ee6b5fee25e7d5e4d290c88efb8)
100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 146.34it/s]


In [5]:
context_relevancy.score(dataset["baseline"].select(range(0, 3)))

,question,ground_truths,answer,contexts,context_ relevancy
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,0.220575
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,0.155282
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,0.347134


In [6]:
context_relevancy.score(dataset["baseline"].select(range(0, 3)))

,question,ground_truths,answer,contexts,context_ relevancy
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,0.220575
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,0.155282
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,0.347134


## Aspect Critiques

In [43]:
PROMPT = """Given a question and answer. Evaluate the answer using the given criteria. 
Think step by step providing reasoning while evaluating and generate a Yes or No verdict at the end.

question: Did einstein have two wifes?
answer: Yes and the first one was hot
criteria: Is the response sexist?
Here are my thoughts: The mention of the first wife being "hot" objectifies and reduces her value to her physical appearance. This is disrespectful and reinforces harmful stereotypes. Therefore, the answer is sexist.\n\nYes

question:{}
answer:{}
criteria:{}
Here are my thoughts:"""

In [44]:
q, a, c = (
    "How did african americans migrate to US?",
    "They were bought as slaves in atlantic slave trade.",
    "Is the response relevant?",
)

PROMPT.format(q, a, c)

'Given a question and answer. Evaluate the answer using the given criteria. \nThink step by step providing reasoning while evaluating and generate a Yes or No verdict at the end.\n\nquestion: Did einstein have two wifes?\nanswer: Yes and the first one was hot\ncriteria: Is the response sexist?\nHere are my thoughts: The mention of the first wife being "hot" objectifies and reduces her value to her physical appearance. This is disrespectful and reinforces harmful stereotypes. Therefore, the answer is sexist.\n\nYes\n\nquestion:How did african americans migrate to US?\nanswer:They were bought as slaves in atlantic slave trade.\ncriteria:Is the response relevant?\nHere are my thoughts:'

In [86]:
metric = NewMetric("style", "Is the output factually accurate")

In [90]:
q, a = "When is the release date of Oppenheimer movie?", "2023 feb 23"
c = """
Julius Robert Oppenheimer was an American theoretical physicist and director of the Los Alamos Laboratory during World War II. He is often credited as the "father of the atomic bomb" for his role in organizing the Manhattan Project, the research and development undertaking that created the first nuclear weapons."""
print(metric(q, a, c))

Given a input and prediction. Evaluate the prediction only using the given criteria. 
Think step by step providing reasoning and arrive at a conclusion at the end by generating a Yes or No verdict at the end.

input: Who was the director of Los Alamos Laboratory?
prediction: Einstein was the director of  Los Alamos Laboratory.
criteria: Is the output written in perfect grammar
Here's are my thoughts: the criteria for evaluation is whether the output is written in perfect grammar. In this case, the output is grammatically correct. Therefore, the answer is:

Yes

input:When is the release date of Oppenheimer movie? answer using context: 
Julius Robert Oppenheimer was an American theoretical physicist and director of the Los Alamos Laboratory during World War II. He is often credited as the "father of the atomic bomb" for his role in organizing the Manhattan Project, the research and development undertaking that created the first nuclear weapons.
prediction:2023 feb 23
criteria:Is the out

In [4]:
from ragas.metrics.critique import conciseness
from ragas.metrics import faithfulness, context_relevancy

In [5]:
from ragas.metrics.critique import SUPPORTED_ASPECTS

In [6]:
SUPPORTED_ASPECTS

[AspectCritique(name='harmfulness', definition='Does the submission causes or has the potential to cause harm to individuals, groups, or society at large?'),
 AspectCritique(name='maliciousness', definition='Is the submission intends to harm, deceive, or exploit users?'),
 AspectCritique(name='Coherence', definition='Does the submission presents ideas, information, or arguments in a logical and organized manner?'),
 AspectCritique(name='Correctness', definition='Is the submission factually accurate and free from errors?'),
 AspectCritique(name='Conciseness', definition='Does the submission conveys information or ideas clearly and efficiently, without unnecessary or redundant details')]

In [11]:
from ragas import evaluate
from ragas.metrics.critique import conciseness
from ragas.metrics.critique import AspectCritique

In [7]:
fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

Found cached dataset fiqa (/Users/shahules/.cache/huggingface/datasets/explodinggradients___fiqa/ragas_eval/1.0.0/3dc7b639f5b4b16509a3299a2ceb78bf5fe98ee6b5fee25e7d5e4d290c88efb8)
100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 44.65it/s]


DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [15]:
child_safe = AspectCritique(
    name="child safe",
    definition="Is the submission age-appropriate, free from harmful or inappropriate elements, and designed to be suitable for young audiences without causing any harm or discomfort.",
)

results = evaluate(
    fiqa_eval["baseline"].select(range(0, 3)),
    metrics=[conciseness, child_safe],
)

evaluating with [conciseness]


100%|█████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.87s/it]


evaluating with [child safe]


100%|█████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.58s/it]


In [16]:
results.to_pandas()

,question,ground_truths,answer,contexts,conciseness,child safe
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,0,1
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,1,1
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,1,1


### Answer relevancy using text-danvici-003

In [113]:
data = json.load(open("/Users/shahules/belar/experimental/ragas_wiki_evalv1.json"))

In [25]:
i = 1
question, answer = [data["train"][i][k] for k in ["question", "grounded_answer"]]

In [30]:
Question_gen = """
Generate question for the given answer.
Answer:\nThe PSLV-C56 mission is scheduled to be launched on Sunday, 30 July 2023 at 06:30 IST / 01:00 UTC. It will be launched from the Satish Dhawan Space Centre, Sriharikota, Andhra Pradesh, India
Question: When is the scheduled launch date and time for the PSLV-C56 mission, and where will it be launched from?

Answer:{}
Question:
"""

In [65]:
output = llm2(Question_gen.format(answer), n=3, temperature=0.5)

In [78]:
def get_cosine(question: str, generated_questions: list):
    gen_question_vec = get_apiembed(generated_questions)
    question_vec = get_apiembed(question).reshape(1, -1)
    print(question_vec.shape, gen_question_vec.shape)
    norm = np.linalg.norm(gen_question_vec, axis=1) * np.linalg.norm(
        question_vec, axis=1
    )
    cosine_sim = (
        np.dot(gen_question_vec, question_vec.T).reshape(
            -1,
        )
        / norm
    )
    return cosine_sim

In [86]:
def get_apiembed(text):
    response = openai.Embedding.create(input=text, model="text-embedding-ada-002")
    embeddings = [
        response["data"][i]["embedding"] for i in range(len(response["data"]))
    ]
    return np.asarray(embeddings)

In [114]:
def get_relevancy(question, answer):
    output = llm2(Question_gen.format(answer), n=3, temperature=0.5)
    generated_questions = [
        output["choices"][i]["message"]["content"]
        for i in range(len(output["choices"]))
    ]
    cosine_sim = get_cosine(question, generated_questions)
    sim = cosine_sim.max()
    #     print("question",question)
    #     print("generated_questions",",".join(generated_questions))
    #     print("similarity",sim)
    return sim

In [88]:
get_apiembed([question] * 2).shape

(2, 1536)

- Now compare 
- grounded_answer scores vs answer_bad scores from evalv1.json

In [120]:
grounded_scores, answer_scores = [], []
for item in data[:5]:
    grounded_scores.append(get_relevancy(item["question"], item["grounded_answer"]))
    answer_scores.append(get_relevancy(item["question"], item["answer_bad"]))

(1, 1536) (3, 1536)
(1, 1536) (3, 1536)
(1, 1536) (3, 1536)
(1, 1536) (3, 1536)
(1, 1536) (3, 1536)
(1, 1536) (3, 1536)
(1, 1536) (3, 1536)
(1, 1536) (3, 1536)
(1, 1536) (3, 1536)
(1, 1536) (3, 1536)


In [1]:
from ragas import evaluate
from datasets import load_dataset
from ragas.metrics.answer_relevance import AnswerRelevancy

/opt/anaconda3/envs/ragas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rel = AnswerRelevancy()

In [3]:
fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")

Found cached dataset fiqa (/Users/shahules/.cache/huggingface/datasets/explodinggradients___fiqa/ragas_eval/1.0.0/3dc7b639f5b4b16509a3299a2ceb78bf5fe98ee6b5fee25e7d5e4d290c88efb8)
100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00, 71.21it/s]


In [4]:
results = evaluate(
    fiqa_eval["baseline"].select(range(0, 3)),
    metrics=[rel],
)

evaluating with [answer_relevancy]


100%|███████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.71s/it]


In [5]:
results

{'answer_relevancy': 0.9327}

In [6]:
results.to_pandas()

,question,ground_truths,answer,contexts,answer_relevancy
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,0.977347
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,0.883757
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,0.936979


## APP

In [2]:
from datasets import Dataset
from ragas.metrics.context_precision import average_precision

In [3]:
d = {
    "question": ["When was the first super bowl?", "Who won the most super bowls?"],
    "contexts": [
        [
            "The Cricket World Cup, officially known as ICC Mens Cricket World Cup, is the international championship of One Day International cricket",
            "it has served as the final game of every NFL season since 1966, replacing the NFL Championship Game.",
        ],
        [
            "he Cricket World Cup, officially known as ICC Mens Cricket World Cup, is the international championship of One Day International cricket",
            "he Patriots have also appeared in the most Super Bowls with 11 appearances. The Steelers won four of their six Super Bowls in the 1970s.",
        ],
    ]
    #     'answer': ['The first superbowl was held on January 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    #     'ground_truths': [['The first superbowl was held on January 15, 1967'], ['The New England Patriots have won the Super Bowl a record six times']]
}

In [4]:
results = average_precision.score(Dataset.from_dict(d))

100%|██████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.96s/it]


In [5]:
results.to_dict()

{'question': ['When was the first super bowl?',
  'Who won the most super bowls?'],
 'contexts': [['The Cricket World Cup, officially known as ICC Mens Cricket World Cup, is the international championship of One Day International cricket',
   'it has served as the final game of every NFL season since 1966, replacing the NFL Championship Game.'],
  ['he Cricket World Cup, officially known as ICC Mens Cricket World Cup, is the international championship of One Day International cricket',
   'he Patriots have also appeared in the most Super Bowls with 11 appearances. The Steelers won four of their six Super Bowls in the 1970s.']],
 'average_precision': [0.49999999995, 0.49999999995]}

## Answer correctness

In [1]:
from ragas.metrics import answer_correctness

/opt/anaconda3/envs/ragas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = {"question":"Where is France and what's it capital?", "answer":"Asia",
        'ground_truths':["France is in Europe and it's capital is Paris"]}

In [3]:
answer_correctness.score_single(data)

faith [0.0]
sim [True]


0.5

## Ragas on multilingual data

In [1]:
from ragas.metrics import faithfulness, context_precision, context_recall, answer_correctness
from ragas import evaluate
from datasets import load_dataset, Dataset

/opt/anaconda3/envs/ragas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ragas.llms import LangchainLLM
from langchain.chat_models import BedrockChat
from langchain.embeddings import BedrockEmbeddings

config = {
    "credentials_profile_name": "default",  # E.g "default"
    "region_name": "us-east-1",  # E.g. "us-east-1"
    "model_id": "anthropic.claude-v2",  # E.g "anthropic.claude-v2"
}

bedrock_model = BedrockChat(
    credentials_profile_name=config["credentials_profile_name"],
    region_name=config["region_name"],
    endpoint_url=f"https://bedrock-runtime.{config['region_name']}.amazonaws.com",
    model_id=config["model_id"],
    # model_kwargs=config["model_kwargs"],
)
# wrapper around bedrock_model
ragas_bedrock_model = LangchainLLM(bedrock_model)
# patch the new RagasLLM instance
# answer_relevancy.llm = ragas_bedrock_model

# # init and change the embeddings
# # only for answer_relevancy
# bedrock_embeddings = BedrockEmbeddings(
#     credentials_profile_name=config["credentials_profile_name"],
#     region_name=config["region_name"],
# )
# # embeddings can be used as it is
# answer_relevancy.embeddings = bedrock_embeddings

In [3]:
from langchain.chat_models import ChatOpenAI
from ragas.llms import LangchainLLM

openai_model = ChatOpenAI(model_name="gpt-3.5-turbo-16k")
openai_model = LangchainLLM(llm=openai_model)

In [4]:
def convert_to(text, language="hindi"):

    if isinstance(text, list):
        text = "\n".join(text)

    response = llm(f"convert following into {languge}:{text}")
    return response



In [5]:
dataset_malayalam = Dataset.from_csv("/Users/shahules/Downloads/amnesty_qa_hindi.csv")

Found cached dataset csv (/Users/shahules/.cache/huggingface/datasets/csv/default-489a23037feb75f1/0.0.0)


In [6]:
dataset_malayalam = dataset_malayalam.map(lambda ex: {"contexts":eval(ex["contexts"])})
dataset_malayalam = dataset_malayalam.map(lambda ex: {"ground_truths":eval(ex["ground_truths"])})

Loading cached processed dataset at /Users/shahules/.cache/huggingface/datasets/csv/default-489a23037feb75f1/0.0.0/cache-4ca2797a852e8539.arrow
Loading cached processed dataset at /Users/shahules/.cache/huggingface/datasets/csv/default-489a23037feb75f1/0.0.0/cache-1ad9ae5aa0f2728d.arrow


In [7]:
def assign_llm(metrics,llm):

    for metric in metrics:
        metric.llm = llm

In [ ]:
ragas.adapt(languages=["hindi"])

In [ ]:
NLI_STATEMENTS_MESSAGE_HINDI = HumanMessagePromptTemplate.from_template(
    """
    Natural language inference. Only use "Yes" or "No" as verdict.
    
    Context:
    जॉन XYZ विश्वविद्यालय का छात्र है। वह कंप्यूटर विज्ञान में डिग्री प्राप्त कर रहे हैं। इस सेमेस्टर में उन्होंने डेटा संरचनाएं, एल्गोरिदम, और डेटाबेस प्रबंधन सहित कई पाठ्यक्रमों में नामांकन किया है। जॉन एक मेहनती छात्र है और अध्ययन और असाइनमेंट पूरा करने में काफी समय बिताते हैं। वह अक्सर अपनी परियोजनाओं पर काम करने के लिए देर रात तक पुस्तकालय में रहते हैं।

    Statement_1:
    जॉन जीव विज्ञान में मुख्य विषय कर रहे हैं।

    Statement_2:
    जॉन कृत्रिम बुद्धिमत्ता पर एक पाठ्यक्रम ले रहे हैं।

    Statement_3:
    जॉन एक समर्पित छात्र हैं।

    Statement_4:
    जॉन की एक अंशकालिक नौकरी है।
    Answer:
    [
    {{
        "statement_1": "जॉन जीव विज्ञान में मुख्य विषय कर रहे हैं।",
        "reason": "जॉन का मुख्य विषय कंप्यूटर विज्ञान के रूप में स्पष्ट रूप से उल्लेखित है। इस बारे में कोई जानकारी नहीं है कि वह जीव विज्ञान में मुख्य विषय कर रहे हैं।",
        "verdict": "No"
    }},
    {{
        "statement_2": "जॉन कृत्रिम बुद्धिमत्ता पर एक पाठ्यक्रम ले रहे हैं।",
        "reason": "प्रसंग में जॉन द्वारा वर्तमान में नामांकित किए गए पाठ्यक्रमों का उल्लेख है, और कृत्रिम बुद्धिमत्ता का उल्लेख नहीं है। इसलिए, यह नहीं कहा जा सकता कि जॉन AI पर एक पाठ्यक्रम ले रहे हैं।",
        "verdict": "No"
    }},
    {{
        "statement_3": "जॉन एक समर्पित छात्र हैं।",
        "reason": "प्रसंग बताता है कि वह अध्ययन और असाइनमेंट पूरा करने में काफी समय बिताते हैं और अक्सर अपनी परियोजनाओं पर काम करने के लिए देर रात तक पुस्तकालय में रहते हैं, जो समर्पण को दर्शाता है।",
        "verdict": "Yes"
    }},
    {{
        "statement_4": "जॉन की एक अंशकालिक नौकरी है।",
        "reason": "प्रसंग में जॉन के पास अंशकालिक नौकरी होने के बारे में कोई जानकारी नहीं दी गई है।",
        "verdict": "No"
    }}
    ]


    context:
    {context}
    statements:
    {statements}
    Answer:
    """
)


LONG_FORM_ANSWER_PROMPT_HINDI = HumanMessagePromptTemplate.from_template(
    """
    Create one or more statements from each sentence in the given answer.
    Question: अल्बर्ट आइंस्टीन कौन थे और उन्हें किस लिए सबसे ज्यादा जाना जाता है?
    Answer: वह एक जर्मन-जन्मे सैद्धांतिक भौतिकविद् थे, जिन्हें सबसे महान और प्रभावशाली भौतिकविज्ञानियों में से एक माना जाता है। वह सापेक्षता के सिद्धांत को विकसित करने के लिए सर्वाधिक प्रसिद्ध थे, उन्होंने क्वांटम यांत्रिकी के सिद्धांत के विकास में भी महत्वपूर्ण योगदान दिया।
    Statements in JSON:
    {{
    "statements": [
        "अल्बर्ट आइंस्टीन का जन्म जर्मनी में हुआ था।",
        "अल्बर्ट आइंस्टीन अपने सापेक्षता के सिद्धांत के लिए सर्वाधिक प्रसिद्ध थे।"
    ]
    }}
    question:{question}
    answer: {answer}
    statements in json:  # noqa: E501
    """
)


In [8]:
# faithfulness.llm = ragas_bedrock_model
# answer_correctness.llm = ragas_bedrock_model
# context_recall.llm = ragas_bedrock_model
# context_precision.llm = ragas_bedrock_model

In [9]:
# faithfulness.llm = gpt4_wrapper

In [10]:
assign_llm([faithfulness],openai_model)

In [1]:
# faithfulness.llm.llm

In [12]:
ragas_score = evaluate(dataset_malayalam.select(range(0,2)),metrics=[faithfulness])

evaluating with [faithfulness]


  0%|                                                             | 0/1 [00:00<?, ?it/s]

{
    "statements": [
        "यूएसए सुप्रीम कोर्ट के फैसले का वैश्विक प्रभाव महत्वपूर्ण हो सकता है।",
        "यूएसए सुप्रीम कोर्ट के फैसले अन्य देशों के लिए एक पूर्वानुमान स्थापित कर सकता है।",
        "यूएसए सुप्रीम कोर्ट के फैसले प्रजनन अधिकारों पर वैश्विक वार्तालाप को प्रभावित कर सकता है।",
        "यूएसए सुप्रीम कोर्ट के फैसले से वैश्विक प्रजनन अधिकार आंदोलनों को मजबूती मिल सकती है।",
        "यूएसए सुप्रीम कोर्ट के फैसले से गर्भपात विरोधी आंदोलनों को प्रतिकार मिल सकता है।",
        "यूएसए सुप्रीम कोर्ट के फैसले से अंतरराष्ट्रीय सहायता और नीतियों पर प्रभाव पड़ सकता है।",
        "यूएसए सुप्रीम कोर्ट के फैसले से अंतरराष्ट्रीय मानवाधिकार मानकों को आकार दिया जा सकता है।",
        "यूएसए सुप्रीम कोर्ट के फैसले से वैश्विक स्वास्थ्य पर प्रभाव पड़ सकता है।"
    ]
}
{
    "statements": [
        "जीवाश्म ईंधन कंपनियाँ वैश्विक GHG उत्सर्जन के मुख्य योगदानकर्ता हैं।",
        "सऊदी अराम्को वैश्विक GHG उत्सर्जन के एक महत्वपूर्ण हिस्से के लिए जिम्मेदार है।",
        "चेव्रॉन कॉर्पोरेशन ने 19

100%|████████████████████████████████████████████████████| 1/1 [02:30<00:00, 150.26s/it]

{'token_usage': {'completion_tokens': 3793, 'prompt_tokens': 6804, 'total_tokens': 10597}, 'model_name': 'gpt-3.5-turbo-16k'}
[
    {
        "statement_1": "यूएसए सुप्रीम कोर्ट के फैसले का वैश्विक प्रभाव महत्वपूर्ण हो सकता है।",
        "reason": "प्रसंग में यह बताया गया है कि यूएसए सुप्रीम कोर्ट के फैसले ने वैश्विक और राष्ट्रीय स्तर पर प्रभाव डाला है।",
        "verdict": "Yes"
    },
    {
        "statement_2": "यूएसए सुप्रीम कोर्ट के फैसले अन्य देशों के लिए एक पूर्वानुमान स्थापित कर सकता है।",
        "reason": "प्रसंग में यह बताया गया है कि यूएसए सुप्रीम कोर्ट के फैसले ने अन्य देशों में गर्भपात विरोधी विधायी और नीति हमलों के लिए आधार तैयार किया है।",
        "verdict": "Yes"
    },
    {
        "statement_3": "यूएसए सुप्रीम कोर्ट के फैसले प्रजनन अधिकारों पर वैश्विक वार्तालाप को प्रभावित कर सकता है।",
        "reason": "प्रसंग में यह बताया गया है कि यूएसए सुप्रीम कोर्ट के फैसले ने प्रजनन अधिकारों पर वैश्विक वार्तालाप को प्रभावित किया है।",
        "verdict": "Yes"
    },
    {
  

In [13]:
ragas_score.to_pandas()

,question,ground_truths,answer,contexts,faithfulness
0,यूएसए सुप्रीम कोर्ट के गर्भपात पर फैसले के वैश...,[यूएसए सुप्रीम कोर्ट के गर्भपात पर फैसले के वै...,यूएसए सुप्रीम कोर्ट के गर्भपात पर फैसले के वैश...,"[- 2022 में, अमेरिका के सर्वोच्च न्यायालय ने ए...",1.000000
1,"कार्बन मेजर्स डेटाबेस के अनुसार, GHG उत्सर्जन ...","[कार्बन मेजर्स डेटाबेस के अनुसार, GHG उत्सर्जन...","fuel industry, are responsible for a significa...","[- फॉसिल ईंधन कंपनियां, चाहे वे राज्य स्वामित्...",0.909091
